In [ ]:
import os, sys, json
import numpy as np
import torch
sys.path.append('..')

from src.config import ModelParams
from src.steady_states import solve_flexprice_sss, export_rbar_tensor
from src.io_utils import ensure_dir, save_json

ARTIFACTS_ROOT = os.path.join("..", "artifacts")

params = ModelParams(device="cpu", dtype=torch.float32)

flex = solve_flexprice_sss(params)
rbar = export_rbar_tensor(params, flex)

print("=== FLEX SSS (by regime) ===")
for s in [0,1]:
    print(f"Regime {s}:")
    for k,v in flex.by_regime[s].items():
        print(f"  {k:>18s}: {v}")

print("\n=== rbar_by_regime ===")
print(rbar)

out_dir = os.path.join(ARTIFACTS_ROOT, "flex")
ensure_dir(out_dir)
save_json(os.path.join(out_dir, "sss.json"), {
    "policy": "flex",
    "by_regime": flex.by_regime,
    "rbar_by_regime": rbar.detach().cpu().tolist(),
})
print(f"Saved flex SSS to: {os.path.join(out_dir, 'sss.json')}")
